# Introduction

Aujourd'hui la plupart des sites web utilisent du JavaScript pour rendre le site plus dynamique et agréable, mais aussi pour afficher la donnée. 

Pour ces sites, la première méthode abordée n'est pas efficace, car de nombreuses requêtes sont utilisées pour se connecter, afficher la donnée, naviguer, il faudrait prendre en compte toutes ces requêtes et les assembler pour simuler le comportement du site. 

**C'est le comportement de votre navigateur**, il compile toutes les interactions, les requêtes et génère du HTML permettant d'afficher l'interface graphique d'un site web.

Pour palier ce problème, on peut laisser le navigateur faire son travail et contrôler sa surcouche logicielle. Pour cela, on peut prendre le contrôle d'un navigateur comme Chrome, Edge, Firefox ou Safari depuis un script Python. 

Pour cela, nous allons utiliser un package Python `Selenium` https://selenium-python.readthedocs.io/ qui permet d'instancier un navigateur et de le contrôler. Sélénium est beaucoup utilisé pour générer des tests automatiques de sites web. 

On peut trouver les drivers des différents browser : 
- Chrome:	https://googlechromelabs.github.io/chrome-for-testing/#stable (bien télécharger le "chromedriver" et non pas chrome)
- Firefox:	https://github.com/mozilla/geckodriver/releases
- Safari:	https://webkit.org/blog/6900/webdriver-support-in-safari-10/

**Depuis les dernières versions de Selenium, il n'est plus néccessaires de télécharger les driver des browsers car Selenium les manage**

In [54]:
from selenium import webdriver


In [55]:
chrome = webdriver.Chrome()

Vous devez voir apparaître une nouvelle instance de votre navigateur.
Chrome vous demande depuis peu de selectionner votre navigateur par défaut. Etant donné que notre but est de tout controller depuis notre code, on peut bypass cette demande en ajoutant des options au webdriver de Chrome

In [56]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--disable-search-engine-choice-screen")

chrome = webdriver.Chrome(options=chrome_options)

Maintenant, vous avez le controlle du navigateur depuis Python. Si vous voulez accéder à une page :

In [57]:
chrome.get("https://books.toscrape.com/")

Allez maintenant voir votre fenêtre. Vous êtes sur un faux site de vente de livre. Ce site est spécialement concu pour être scrapé et comprendre les bases, mais en réalité scraper un site est un peu plus complexe, comme vous allez vous en rendre compte dans la section 2 sur Scrapy.

Vous pouvez maintenant vous déplacer dans la page. Pour cela, il va falloir inspecter le code source de la page.

Il existe de nombreuse méthode pour récupérer les éléments par exemple `find_element` qui permet de récupérer un élément grâce au texte affiché. Ici, si on veut récupérer cliquer sur la section `Travel` pour récupérer les livres de voyages

In [58]:
from selenium.webdriver.common.by import By
link = chrome.find_element(By.LINK_TEXT, "Travel")
type(link)

selenium.webdriver.remote.webelement.WebElement

Si on veut entrer dans le lien, il suffit d'appeler la méthode `click`.

In [59]:
link.click()

Vous êtes arrivé sur la page des livres de voyage. Maintenant nous allons récupérer toutes les livres. On remarque que tous les livres ont une class HTML nommée `product_pod`.

In [60]:
all_widgets = chrome.find_elements(By.CLASS_NAME, "product_pod")
type(all_widgets), len(all_widgets)

(list, 11)

On obtient une liste d'éléments décrits par cette class. Il y en a 11, un pour chaque livre. Pour récupérer le titre du premier livre: 

In [61]:
first_book = all_widgets[0]
first_book_title = first_book.find_element(By.CSS_SELECTOR, "h3")
first_book_title.text

"It's Only the Himalayas"

Si vous êtes fan de JavaScript vous pouvez même injecter du code JS dans le navigateur depuis Python pour exécuter des opérations complexes. 

In [62]:
js_script = """
const class_name = 'DSIA'
alert(`Hi from ${class_name}`)
"""

In [63]:
chrome.execute_script(js_script)

# Exercice

## Exercice 1

Extraire le prix des livres :

In [65]:
def extract_price(book_web_element):
    # raise NotImplementedError('Need to be implemented')
    book_list = []
    for book in book_web_element:
        book_price = book.find_element(By.CSS_SELECTOR, ".price_color").text       
        book_list.append(book_price)
        
    return book_list


book_prices = extract_price(all_widgets)
print(book_prices)

['£45.17', '£49.43', '£48.87', '£36.94', '£37.33', '£44.34', '£30.54', '£56.88', '£23.21', '£38.95', '£26.08']


Extraire l'image des livres: 

In [66]:
def extract_images(book_web_elements):
    image_list = []
    for book in book_web_elements:
            book_image = book.find_element(By.CSS_SELECTOR, "img").get_attribute("src")
            image_list.append(book_image)
    return image_list

book_images = extract_images(all_widgets)
print(book_images)

['https://books.toscrape.com/media/cache/27/a5/27a53d0bb95bdd88288eaf66c9230d7e.jpg', 'https://books.toscrape.com/media/cache/57/77/57770cac1628f4407636635f4b85e88c.jpg', 'https://books.toscrape.com/media/cache/9a/7e/9a7e63f12829df4b43b31d110bf3dc2e.jpg', 'https://books.toscrape.com/media/cache/d5/bf/d5bf0090470b0b8ea46d9c166f7895aa.jpg', 'https://books.toscrape.com/media/cache/98/c2/98c2e95c5fd1a4e7cd5f2b63c52826cb.jpg', 'https://books.toscrape.com/media/cache/4e/15/4e15150388702ebca2c5a523ac270539.jpg', 'https://books.toscrape.com/media/cache/76/de/76de41867f323d7f1f4fbe2fdfc1b2ba.jpg', 'https://books.toscrape.com/media/cache/db/46/db46159b05faa5d95262112bf9c29ddd.jpg', 'https://books.toscrape.com/media/cache/e0/4f/e04f8eda2a2fa947aec17640202d9ab0.jpg', 'https://books.toscrape.com/media/cache/06/81/0681530a7bc301caf5c3257e1b0f0750.jpg', 'https://books.toscrape.com/media/cache/d7/0f/d70f7edd92705c45a82118c3ff6c299d.jpg']


Extraire le nombre d'étoiles d'un livre: 

In [71]:
def extract_stars(book_web_elements):
    stars_list = []
    for book in book_web_elements:
            book_star = book.find_element(By.CSS_SELECTOR, "p.star-rating").get_attribute("class")
            stars_list.append(book_star)
    return stars_list

book_stars = extract_stars(all_widgets)
print(book_stars)

['star-rating Two', 'star-rating Four', 'star-rating Three', 'star-rating Two', 'star-rating Three', 'star-rating Two', 'star-rating One', 'star-rating Four', 'star-rating One', 'star-rating Three', 'star-rating Five']


In [73]:
chrome.close()

InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x0000000104957ac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x0000000104950314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x00000001043b8324 cxxbridge1$string$len + 88824
3   chromedriver                        0x00000001043f05d0 cxxbridge1$string$len + 318884
4   chromedriver                        0x000000010441b244 cxxbridge1$string$len + 494104
5   chromedriver                        0x0000000104416724 cxxbridge1$string$len + 474872
6   chromedriver                        0x0000000104415a54 cxxbridge1$string$len + 471592
7   chromedriver                        0x0000000104389e44 chromedriver + 89668
8   chromedriver                        0x0000000104922e08 cxxbridge1$str$ptr + 3435328
9   chromedriver                        0x0000000104926120 cxxbridge1$str$ptr + 3448408
10  chromedriver                        0x0000000104